# Benchmark

In this notebook we will make use of the benchmark module included in PhotoHolmes in order to benchmark Fusion with some of the included metrics in PhotoHolmes

In [1]:
# Imports from PhotoHolmes
from photoholmes.benchmark import Benchmark
from photoholmes.datasets.factory import DatasetFactory
from photoholmes.methods.factory import MethodFactory
from photoholmes.metrics.factory import MetricFactory

In [2]:
# The benchamrk module 
device = "mps"
benchmark = Benchmark(
            device=device,
            verbose=1,
            save_metrics=True,
        )

#benchmark.run()

In [3]:
# CocoGlide dataset
import glob
import os
from photoholmes.datasets.base import BaseDataset
from torch import Tensor

class CocoGlide(BaseDataset):
    IMAGE_EXTENSION = ".png"
    MASK_EXTENSION = ".png"

    def _get_paths(self, dataset_path: str, tampered_only: bool = False):
        image_paths = glob.glob(os.path.join(dataset_path, "fake", f"*{self.IMAGE_EXTENSION}"))
        if not self.tampered_only:
            image_paths += glob.glob(os.path.join(dataset_path, "real", f"*{self.IMAGE_EXTENSION}"))
        print(len( image_paths ))

        mask_paths = [self._get_mask_path(image_path) for image_path in image_paths]
        
        return image_paths, mask_paths
        

    def _get_mask_path(self, image_path: str) -> str:
        if "glide" not in image_path: # fake images all start with "glide"
            return None
        
        image_id = image_path.split("_")[-2]
        return glob.glob(os.path.join(self.dataset_path, "mask", f"*_{image_id}_*{self.MASK_EXTENSION}"))[0]

    def _binarize_mask(self, mask_image: Tensor) -> Tensor:
        return (mask_image[0,:,:] == 255).float()


In [4]:
# Metrics
metrics = MetricFactory.load(
            [
                "auroc",
                "f1_weighted_v1",
                "iou_weighted_v1",
                "mcc_weighted_v1",
            ]
        )


In [5]:
# Fusion
import torch
from photoholmes.methods.base import BenchmarkOutput, BaseTorchMethod
from photoholmes.preprocessing import PreProcessingPipeline, ZeroOneRange
import torchvision.transforms.functional as TF

from models.cmnext_conf import CMNeXtWithConf
from models.modal_extract import ModalitiesExtractor
from configs.cmnext_init_cfg import _C as config, update_config


class MMFusion(BaseTorchMethod):

    def __init__(self, exp: str = "experiments/ec_example_phase2.yaml", checkpoint: str = None, device: str = "cpu"):
        super().__init__()
        cfg = update_config(config, exp)

        self.modal_extractor = ModalitiesExtractor(
            cfg.MODEL.MODALS[1:], cfg.MODEL.NP_WEIGHTS
        )
        self.model = CMNeXtWithConf(cfg.MODEL)

        ckpt = torch.load(checkpoint, map_location=device)
        self.model.load_state_dict(ckpt['state_dict'])
        self.modal_extractor.load_state_dict(ckpt["extractor_state_dict"])

        self.eval()

    @torch.no_grad()
    def predict(self, image):
        if image.ndim == 3:
            image = image.unsqueeze(0)

        modals = self.modal_extractor(image)

        images_norm = TF.normalize(image, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        inp = [images_norm] + modals

        anomaly, confidence, detection = self.model(inp)
        
        heatmap = torch.nn.functional.softmax(anomaly, dim=1)[:, 1, :, :].squeeze()
        return heatmap, confidence, detection
    
    def benchmark(self, image) -> BenchmarkOutput:
        heatmap, _, detection = self.predict(image)
       # heatmap = torch.tile(heatmap, (3, 1, 1))
        return BenchmarkOutput(heatmap=heatmap, detection=detection, mask=None)


mmfusion_preprocessing = PreProcessingPipeline(
    transforms=[
        ZeroOneRange()
    ],
    inputs=["image"],
    outputs_keys=["image"]
)

/Users/julietaumpierrez/Desktop/MMFusion-IML/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Benchmarking

dataset = CocoGlide('dataset/', preprocessing_pipeline= mmfusion_preprocessing, tampered_only=True)
method = MMFusion("experiments/ec_example_phase2.yaml", "ckpt/model_zoo/early_fusion_detection.pth")
benchmark.run(
            method=method,
            dataset=dataset,
            metrics=metrics,
        )


WARNING - The load attribute and the preprocessing pipeline inputs do not match. Using the preprocessing pipeline inputs: ['image']


512


INFO - Noiseprint++ weights: pretrained/noiseprint/np++.pth
INFO - Currently training for detection
INFO - Loading Model: ec_example_phase2, with backbone: MixCMNeXtMHSA-B2
INFO - Loading pretrained module: pretrained/segformer/mit_b2.pth
INFO - Using device: mps
INFO - --------------------------------------------------------------------------------
INFO - --------------------------------------------------------------------------------
INFO - Running the benchmark
INFO - --------------------------------------------------------------------------------
INFO - --------------------------------------------------------------------------------
INFO - Benchmark configuration:
INFO -     Method: MMFusion
INFO -     Dataset: CocoGlide
INFO -     Metrics:
INFO -        - BinaryAUROC
INFO -        - F1_weighted_v1
INFO -        - IoU_weighted_v1
INFO -        - MCC_weighted_v1
INFO -     Output path: output/mmfusion/cocoglide
INFO -     Save method outputs: True
INFO -     Save metrics: True
INFO 

{'heatmap': {'heatmapBinaryAUROC': tensor(0.4862, device='mps:0'),
  'heatmapF1_weighted_v1': tensor(0.2604, device='mps:0'),
  'heatmapIoU_weighted_v1': tensor(0.1603, device='mps:0'),
  'heatmapMCC_weighted_v1': tensor(-0.0106, device='mps:0')},
 'detection': {'detectionBinaryAUROC': tensor(0., device='mps:0'),
  'detectionF1_weighted_v1': tensor(0.5068, device='mps:0'),
  'detectionIoU_weighted_v1': tensor(0.3397, device='mps:0'),
  'detectionMCC_weighted_v1': tensor(0., device='mps:0')}}

In [10]:
# Now we can see how fast the benchmark goes if we already have outputs but we want to compute new metrics
metrics = MetricFactory.load(["tpr"])
benchmark.run(
            method=method,
            dataset=dataset,
            metrics=metrics,
        )

INFO - Using device: cpu
INFO - --------------------------------------------------------------------------------
INFO - --------------------------------------------------------------------------------
INFO - Running the benchmark
INFO - --------------------------------------------------------------------------------
INFO - --------------------------------------------------------------------------------
INFO - Benchmark configuration:
INFO -     Method: DQ
INFO -     Dataset: CocoGlide
INFO -     Metrics:
INFO -        - BinaryRecall
INFO -     Output path: output/dq/cocoglide
INFO -     Save method outputs: True
INFO -     Save metrics: True
INFO -     Device: cpu
INFO -     Load existing outputs: False
INFO -     Verbosity: INFO
INFO - --------------------------------------------------------------------------------
INFO - --------------------------------------------------------------------------------
Processing Images:   0%|          | 0/1024 [00:00<?, ?it/s]/Users/julietaumpierrez/D

{'heatmap': {'heatmapBinaryRecall': tensor(0.0024)}}